# Data structures

The main data structures are:

- the `Array` object, which can be thought of as a numpy `ndarray` with a physical unit
- the `Dataset` object, which acts as a dictionary of Arrays, and can also contain additional metadata

This notebook aims to give both a description of each structure,
and show how they can be used efficiently to manipulate and explore your simulation data.
They aim to have an API which is close to the Python dict and the numpy `ndarray`.

In [ ]:
import osyris
import numpy as np

We load a data output from a star formation simulation

In [ ]:
data = osyris.load(71, scale="au", path="osyrisdata")

## The `Dataset` class

The `load` function returns a `Dataset` object that has a `__str__` representation,
which list all the contents of `data` in an easy manner:

In [ ]:
data

The `Dataset` class aims to behave very similarly to a Python `dict`.
To access one element of `data`, we index it by using the variable names

In [ ]:
data["density"]

Each entry in the `Dataset` is an `Array` object, which will be described in detail below.

The `Dataset` also provides standard dictionary iterators, which can be used to loop over its contents:

In [ ]:
data.keys()

In [ ]:
data.values()

Finally, it is possible to store additional metadata under the `meta` property

In [ ]:
data.meta

## The `Array` object

Each entry in the `data` dictionary is an `Array` object:

In [ ]:
type(data["density"])

Its string representation lists the array's key in its parent `Dataset`,
the minimum and maximum values in the array, its physical unit, and the number of elements in the array.

In [ ]:
data["density"]

An `Array` can basically be thought of as a `numpy` array with a physical unit: 

In [ ]:
data["density"].values

In [ ]:
data["density"].unit

Operations you would normally perform on a numpy array, such as slicing, are supported:

In [ ]:
data["density"][101:255]

Note that this returns a view onto the original data, instead of making a copy.

Using `numpy`'s `array_function` and `array_ufunc` protocols, `Array` also supports most `numpy` operations, e.g.

In [ ]:
np.log10(data["density"])

In [ ]:
np.sum(data["density"])

Note that in these cases, a new `Array` is returned, and they are not attached to any `Dataset`,
and therefore do not have a name.
To insert a new `Array` into a `Dataset`, simply use the dictionary syntax

In [ ]:
data["log_rho"] = np.log10(data["density"])
data

### Example: find the system center

A simple way to find the centre of our protostellar system is to use the coordinate of the cell with the highest density in the simulation.

In [ ]:
ind = np.argmax(data["density"])
center = data["xyz"][ind.values]
center

## Array arithmetic and units

Units are automatically handled (and conversions carried out) when performing arithmetic on arrays.
For instance, we want to compute a new quantity which represents the mass inside each cell.

The data density is in `g / cm**3`

In [ ]:
data["density"]

while the cell size is in astronomical units

In [ ]:
data["dx"]

However, we can still multiply them together

In [ ]:
data["mass"] = data["density"] * (data["dx"]**3)
data["mass"]

The conversion between `au` and `cm` is automatically handled by first converting both operands to their base units, before performing the operation.

This helps to free mental capacity and allows the user to focus on what is important: **doing science**.

### Manual unit conversions

Sometimes, it is useful to convert from CGS units to other base units:

In [ ]:
data["mass"].to("msun")
data["mass"]

### Units also provide safety

Physical units also provide a certain level of safety around operations.
By assigning some quantities to intermediate variables,
it is often easy to lose track of the exact quantity (or at least its dimensionality) that a variable represents.

Physical units can prevent performing operations on mismatching quantities:

In [ ]:
try:
    data["density"] + data["mass"]
except:
    print("DimensionalityError: Cannot add 'density' and 'mass', mismatching units.")

Physical units can also help find errors in an analysis workflow,
when looking at a final value for a computed star formation rate for example,
and realising that the final unit represents a quantity per unit volume,
while the user was trying to calculate an integrate rate.

### Automatic broadcast

Arrays can either represent a scalar quantity (e.g. density) or a vector quantity (e.g. velocity).
When performing arithmetic that involves both scalars and vectors, an automatic broadcast mechanism handles the operations:

In [ ]:
data["momentum"] = data["density"] * data["velocity"]
data["momentum"]

### Operations with floats

Operations with floats are also supported:

In [ ]:
data["density"] * 1.0e5